# 2: Spherical wind
-------------------


In [1]:
import numpy             as np
import scipy             as sp
import matplotlib.pyplot as plt
import magritte.tools    as tools
import magritte.setup    as setup
import magritte.core     as magritte

from astropy import units, constants

In [2]:
# Deifne names for the data files
model_file = 'output/magritte_spherical_wind.hdf5'
lamda_file = 'data/co.txt'

If necessary, download the line data file form the [LAMDA database](https://home.strw.leidenuniv.nl/~moldata/).

In [3]:
!wget "https://home.strw.leidenuniv.nl/~moldata/datafiles/co.dat" --output-document $lamda_file

/bin/bash: wget: command not found


You will modify two parameters to find the best fit model: the mass-loss rate and the power-law exponent of the temperature profile

In [4]:
# Define the mass-loss rate (Mdot) and temperature profile exponent (epsilon)
Mdot     = (5e-6  * units.M_sun/units.yr).si.value
epsilon  =  0.5


In [5]:
# Now define the other parameters we will need
distance = (740     * units.pc            ).si.value  # distance to WX Psc
r_star   = (520     * constants.R_sun     ).si.value  # stellar radius
r_in     =  1.7     * r_star                          # radius of the inner edge of the circumstellar envelope
r_out    = (1.1e+17 * units.cm            ).si.value  # radius of the outer edge of the circumstellar envelope
v_inf    = (19.8    * units.km/units.s    ).si.value  # expansion velocity of the envelope
T_star   = (2750    * units.K             ).si.value  # effective temperature of the star
T_in     = (1000    * units.K             ).si.value  # temperature at r_in = dust condensation temperature
xCO      =  1.0e-4                                    # fractional CO abundance (relative to H2)
v_turb   = (1.5e+0  * units.km   /units.s ).si.value  # turbulent velocity

# Define the number and array of radial points
npoints  = 128
rs = np.logspace(np.log10(r_in), np.log10(r_out), npoints, dtype=np.float64)

# Define the other arrays
v_r = v_inf  * np.ones_like(rs)                 # expansion velocity
trb = v_turb * np.ones_like(rs)                 # turbulent velocity
tmp = T_in * (r_in / rs)**epsilon             # temperature
rho = Mdot / (4.0 * np.pi * rs**2 * v_r)        # density
nH2 = (constants.N_A.si.value / 2.02e-3) * rho  # H2 density
nCO = xCO * nH2                                 # CO density

In [6]:
# Set up a spherically symmetric Magritte model with the parameters defined above
model = magritte.Model ()
model.parameters.set_spherical_symmetry(True)
model.parameters.set_model_name        (model_file)
model.parameters.set_dimension         (1)
model.parameters.set_npoints           (npoints)
model.parameters.set_nrays             (12)
model.parameters.set_nspecs            (3)
model.parameters.set_nlspecs           (1)
model.parameters.set_nquads            (5)

model.geometry.points.position.set([[r,                      0, 0] for r in rs])
model.geometry.points.velocity.set([[v/constants.c.si.value, 0, 0] for v in v_r])

model.chemistry.species.abundance = [[CO, H2, 0.0] for (CO, H2) in zip(nCO, nH2)]
model.chemistry.species.symbol    = ['CO', 'H2', 'e-']

model.thermodynamics.temperature.gas  .set( tmp                         )
model.thermodynamics.turbulence.vturb2.set((trb/constants.c.si.value)**2)

model = setup.set_Delaunay_neighbor_lists (model)
model = setup.set_Delaunay_boundary       (model)
model = setup.set_boundary_condition_CMB  (model)
model = setup.set_rays_spherical_symmetry (model)
model = setup.set_linedata_from_LAMDA_file(model, lamda_file)
model = setup.set_quadrature              (model)

# Write and read model to initialize all variables
model.write()
model.read ()

Writing parameters...
Writing points...
Writing rays...
Writing boundary...
Writing chemistry...
Writing species...
Writing thermodynamics...
Writing temperature...
Writing turbulence...
Writing lines...
Writing lineProducingSpecies...
Writing linedata...
ncolpoar = 2
--- colpoar = 0
Writing collisionPartner...
(l, c) = 0, 0
--- colpoar = 1
Writing collisionPartner...
(l, c) = 0, 1
Writing quadrature...
Writing populations...
Writing radiation...
Writing frequencies...
                                           
-------------------------------------------
  Reading Model...                         
-------------------------------------------
 model file = output/magritte_spherical_wind.hdf5
-------------------------------------------
Reading parameters...
Reading points...
Reading rays...
Reading boundary...
Reading chemistry...
Reading species...
Reading thermodynamics...
Reading temperature...
Reading turbulence...
Reading lines...
Reading lineProducingSpecies...
Reading linedata...


In [7]:
# Initialize model with LTE level populations
model.compute_spectral_discretisation ()
model.compute_inverse_line_widths     ()
model.compute_LTE_level_populations   ()

Computing spectral discretisation...


0

Computing inverse line widths...
Computing LTE level populations...


In [8]:
%%time

# Iterate level populations until statistical equilibrium
model.compute_level_populations_sparse (True, 200)

Starting iteration 0
using ng acceleration? 0
Computing the radiation field...
Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
Tot | Compute Radiation Field : 1.011623 seconds
Tot | Compute Statistical Equilibrium : 0.004464 seconds
Already 28.0678 % converged!
Starting iteration 1
using ng acceleration? 0
Computing the radiation field...
Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
Tot | Compute Radiation Field : 2.026958 seconds
Tot | Compute Statistical Equilibrium : 0.009463 seconds
Already 36.6425 % converged!
Starting iteration 2
using ng acceleration? 0
Computing the radiation field...
Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
Tot | Compute Radiation Field : 3.111301 seconds
Tot | Compute Statistical Equilibrium : 0.014272 seconds
Already 55.5259 % converged!
Starting iteration 3
using ng acceleration? 1
Ng acceleration max order: 

KeyboardInterrupt: 

--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
Tot | Compute Radiation Field : 131.980825 seconds
Tot | Compute Statistical Equilibrium : 0.598555 seconds
Already 96.5701 % converged!
Starting iteration 132
using ng acceleration? 1
Ng acceleration max order: 3 used order: 2
Already 97.0846 % converged!
Starting iteration 132
using ng acceleration? 0
Computing the radiation field...
Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
Tot | Compute Radiation Field : 133.057719 seconds
Tot | Compute Statistical Equilibrium : 0.603809 seconds
Already 96.6082 % converged!
Starting iteration 133
using ng acceleration? 0
Computing the radiation field...
Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
Tot | Compute Radiation Field : 134.105338 seconds
Tot | Compute Statistical Equilibrium : 0.608323 seconds
Already 96.8369 % converged!
Starting iteration 134
using ng acceleration? 0
Computing the radiation f

In [9]:
# Now plot the output model spectra against the data (***currently in Tmb (K) but can be converted to Jy if that's easier!), and calculate the chi2 value for goodness of fit


Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
Tot | Compute Radiation Field : 186.743582 seconds
Tot | Compute Statistical Equilibrium : 0.843789 seconds
Already 96.9703 % converged!
Starting iteration 186
using ng acceleration? 1
Ng acceleration max order: 3 used order: 2
Already 97.5229 % converged!
Starting iteration 186
using ng acceleration? 0
Computing the radiation field...
Computing radiation field...
--- rr = 0
--- rr = 1
--- rr = 2
--- rr = 3
--- rr = 4
--- rr = 5
